In [1]:
from argparse import ArgumentParser
from BitcoinAPI import BitcoinAPI
from enum import Enum
import networkx as nx
from networkx.readwrite import json_graph
import json
import statistics
import yaml
import pandas as pd
import requests
from argparse import ArgumentParser
from multiprocessing import Process, Pipe
import traceback


In [2]:
data = pd.read_csv('../data/data.csv')
data.head()

,title,category,address
0,Hydra Market,market,1BCWMwpR4M1nYUuuYe2bmzrNuwGoF9ZAbA
1,Hydra Market,market,1MQBDeRWsiJBf7K1VGjJ7PWEL6GJXMfmLg
2,Hydra Market,market,bc1qj6j6p0jdefl6pvdzx3kx8245yy5mz6q4luhzes
3,Hydra Market,market,1D1ej7zQzywWBDNXKNYpmH7Hso2U9koDG4
4,Hydra Market,market,1A3iYY4c3dkgNYGewzYzr7EsqfBuWXibGo


In [10]:
def get_address_txs(address):
    txs = []
    # return list txs of all transactions for address
    # if a has more than 25 txs, we need to fetch all of them
    BASE_URL = 'https://blockstream.info/api/'

    transaction_url = BASE_URL + 'address/' + str(address) + '/txs'
    response = requests.get(transaction_url)
    n = 0
    if response.status_code == 200:
        json_data = response.json()
        n = len(json_data)
        
    while response.status_code == 200 and n>0: 
        for i in range (len(json_data)):
            txs.append(json_data[i])
        url_next_transac =  transaction_url + '/chain/' + str(json_data[len(json_data)-1]['txid'])
        response = requests.get(url_next_transac)
        json_data = response.json()
        n = len(json_data)
    return txs

In [11]:
def get_in_addr(tx_hash):
    BASE_URL = 'https://blockstream.info/api/'
    l_addr_in = [] # list of input addresses
    url = BASE_URL + 'tx/' + str(tx_hash) 
    
    response = requests.get(url)
    if response.status_code == 200:
        json_data = response.json()

        for i in range (len(json_data['vin'])):
            if 'scriptpubkey_address' in json_data['vin'][i]['prevout'].keys() :
                if json_data['vin'][i]['prevout']['scriptpubkey_address'] not in l_addr_in:
                    l_addr_in.append(json_data['vin'][i]['prevout']['scriptpubkey_address'])
    
    return l_addr_in

In [5]:
BASE_URL = 'https://blockstream.info/api/'
transactions = get_address_txs("12sENwECeRSmTeDwyLNqwh47JistZqFmW8")
transactions

[{'txid': 'd5b8e00a9ad97f4460e72f745448519973016ad9c942beed0dabfc9792bc69d9',
  'version': 1,
  'locktime': 0,
  'vin': [{'txid': '8446e3b34bc09e5458fd572c558a99f98795a5b88253694e99c754d81d17e68d',
    'vout': 50,
    'prevout': {'scriptpubkey': '76a914877232bf3e32912576404f0d3b39d15920cc968788ac',
     'scriptpubkey_asm': 'OP_DUP OP_HASH160 OP_PUSHBYTES_20 877232bf3e32912576404f0d3b39d15920cc9687 OP_EQUALVERIFY OP_CHECKSIG',
     'scriptpubkey_type': 'p2pkh',
     'scriptpubkey_address': '1DMB2XhUqr7RCefSixn1pmAGXrjB4gMvm7',
     'value': 887405},
    'scriptsig': '48304502210085e0cfe8b877d7de64f5ab41c4f92736fae62b111cc4f2261039f4ca7c94fd740220757782360fd0bfb355cc6293312498e198c76d8e7773ae788f678e71029bc1d10141047d8a55c8098287f47e3e142070f290fb142a59bff42aad30d92752b7eb1c77b73f2b98daa0b4ed87e2e2d31d5bbbad5013f79d2374bed907361f8f209e14e164',
    'scriptsig_asm': 'OP_PUSHBYTES_72 304502210085e0cfe8b877d7de64f5ab41c4f92736fae62b111cc4f2261039f4ca7c94fd740220757782360fd0bfb355cc6293312498

In [28]:
len(transactions)

144

In [32]:
tx = transactions[0]

In [31]:
transactions[0].keys()

dict_keys(['txid', 'version', 'locktime', 'vin', 'vout', 'size', 'weight', 'fee', 'status'])

In [36]:
tx['vout']

84

In [75]:
tx = transactions[4]
output_adresses = list(map((lambda x:x["scriptpubkey_address"] if "scriptpubkey_address" in x else None),tx["vout"]))
print(len(output_adresses))
if len(output_adresses) <= 10 :
    for addr in output_adresses : 
        txs = get_address_txs(addr)
        print(f"{addr} made {len(txs)} transactions")
        for i in range (len(txs)):
            tx_i  = txs[i]
            inputs = list(map(lambda x:x["txid"],tx_i["vin"]))
            if tx['txid'] in inputs :
                print(f"transaction {tx['txid']} used as an input in {tx_i['txid']}")

1
1JEC8vYP9cEDSu6N6DXkkYd3RaeWAdsCqN made 35 transactions
transaction f4b8759beea5b1e946a0912f56866f1a838cd9e3a5eece8df27a064d742fb024 used as an input in 8f1d3a8ef6b2d4a25d2f499279e01518b4770819ccbc39a765c4c326170c61b3


In [20]:
def recursive_search(tx_hash, depth, depth_max,score):
    BASE_URL = 'https://blockstream.info/api/'
    url = BASE_URL + 'tx/' + str(tx_hash) 
    
    response = requests.get(url)
    if response.status_code == 200:
        tx = response.json()
    
    output_adresses = list(map((lambda x:x["scriptpubkey_address"] if "scriptpubkey_address" in x else None),tx["vout"]))
    print(f"depth {depth}, {len(output_adresses)} output adresses")
    if len(output_adresses) <= 10:
        for addr in output_adresses : 
            score_addr = 0
            if addr in data['address'].values :
                print(f"illegal adress found downstream {addr} at depht {depth}")
                satoshi = 0
                for j in range (len(tx['vout'])):
                    if 'scriptpubkey_address' in tx['vout'][j].keys():
                        if tx['vout'][j]['scriptpubkey_address'] == addr:
                            satoshi = tx['vout'][j]['value']
                            score_addr = satoshi/depth
                            depth = depth_max
                            j = len(tx['vout'])
                                                    
            score = max(score,score_addr)
            

            txs = get_address_txs(addr)
            if txs is None :
                print("none")
            #print(f"{addr} made {len(txs)} transactions")
            for i in range (len(txs)):
                tx_i  = txs[i]
                inputs = list(map(lambda x:x["txid"],tx_i["vin"]))
                if tx['txid'] in inputs :
                    #print(f"{tx['txid']} --> {tx_i['txid']}")
                    if  depth < depth_max:
                        #recursive_search(tx_i['txid'],depth, depth_max, score)
                        score = max(score,recursive_search(tx_i['txid'], depth+1, depth_max, score))

            #print(f"{addr} has not reused {tx['txid']}")
    return score

In [165]:
initial_score = 0
score = recursive_search('0dd0cce2ff8520b049dfb8bf43f63b2a4a735d7d1e84a000d1b330fe95cde144', 1, 3, initial_score)

depth 0, 2 output adresses


KeyboardInterrupt: 

In [180]:
score = recursive_search('a4f7df3b932b5a321a9523135c9cfab39107e8b4c79e7f643cba8ee47e6eb3c3', 1, 3, initial_score)
score

depth 1, 2 output adresses
depth 2, 1 output adresses
depth 3, 1 output adresses
depth 2, 1 output adresses
depth 3, 2 output adresses
illegal adress found downstream


10486007.0

In [194]:
score = recursive_search('e8e1712f5e654f88f2f6ff46193e76b2b7085837a97a3b04eec97de21fb1ada6', 1, 4, initial_score)
score

depth 1, 2 output adresses
['1GZN44id9LqksYN7myh2PqJ26QgMbGmFiW', '1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA']
depth 2, 2 output adresses
['1J9Pjf5SwfaUBUbnY5LbQwDKYTBc8icTzK', '1K6dt8zDRGoDiHEm94n7P91weUsR7bL2QG']
depth 3, 2 output adresses
['18uGHkzu4WvfQuciukbZETgWyQtLV6ccER', '1MNGaQBBXp8cP8V3FeG7aP9xgsTu92A97g']
depth 4, 2 output adresses
['163RbXf7qjAxtwvEKKNcYuMZ7aREoq4K5n', '1Nh3aXoN1nhXXwKeBmdHnusP7L7TXZ4A2t']
depth 4, 249 output adresses
['16GMmUYMX18ZP4sm2tbvkezkMNxDAabQKH', '1342B1zqxMyiVmdpPne3h7EnxbySfbwWGq', '13jZzL7CJfGwunR7SBMb6PXJTgAcGMZ2Vf', '34PBNWDe81nTjUearMog738ZSCF43tMBaq', '1F3J4NSSBeETaoy7VPFpks4MPEEYYhEfBR', '15cKkuRT8NbuHxxoSsf5tfcRvfqdHmk76V', '1ProboismUCP7FwGhwm6Uz6uosy6Zd15d', '3Qk1ZUcmFJEK84KNxoWj1r2yAfUTPsY28H', '1AJV9D5J2nEvstLVb5gtVLR3ujnYXH79jC', '1MN4Rmdtbxv3vyTPGU4FhJ2jb57TBE2RLK', '14CdaZ5yW2b2J2rLvp3bMJnYfqqE9ZKSji', '163WqWc6ZQRAXHpe4RpVYGt2uMAtMjeJnH', '1D8vdpSfSA1EDq4w3wwaZWjyxQJUyfrxhR', '1Svzh4Q79SaZ81ymXH5BTTd2iavZrhpxj', '1HcoPVTsZfg5Ru9LNtwWAe6

depth 3, 2 output adresses
['15TXrz2ftdeFKEx9HNceDEnAxPiKSnf6C8', '13QNFpfQeApEsJ2U1yWUQLFZTg43N4Jau2']
depth 4, 2 output adresses
['17NTuARP1dii8iufZtf4rRhsnnVWzpcYN4', '177pW8jm95DZeuCDLwczXVYAHiQeQSZ3eC']
depth 4, 2 output adresses
['17DJju8hR77ZJ91isS3JM3ShGri44GxRYj', '19npeEcSpQJxWDpQsXgX1cARKNb8QjezfU']
illegal adress found downstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depht 1


10862307.0

In [21]:
score = recursive_search('7660c3c260148a231455c5978762a935b048860bf740da469bf1f247b9bd5518', 1, 3, 0)
score

depth 1, 1 output adresses
depth 2, 2 output adresses
illegal adress found downstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depht 2


1341970.0

In [8]:
score = recursive_search('e377ac9333e4527fa86d4e096525f7ca81e9fd6212237d8601a0ccae916de23c', 1, 3, 0)
score

depth 1, 2 output adresses
depth 2, 2 output adresses
depth 3, 2 output adresses
depth 3, 2 output adresses
depth 2, 2 output adresses
illegal adress found downstream 1LWadswFVXwCoVSKAeo3tuxWKqKr1EWFxR at depht 2


500000.0

In [9]:
score = recursive_search('f99b8281b1e04beaba54bf5529f370195c55db814e91ecb51518960df0522235', 1, 3, 0)
score

depth 1, 2 output adresses
depth 2, 2 output adresses
depth 3, 2 output adresses
depth 2, 2 output adresses
depth 3, 2 output adresses
depth 3, 2 output adresses
illegal adress found downstream 1LWadswFVXwCoVSKAeo3tuxWKqKr1EWFxR at depht 3


333333.3333333333

In [14]:
len(get_address_txs("12M99KTTWAQAK9NRCP2TEXiyLqPgmdCjJk"))

319

In [12]:
def recursive_upstream_search(tx_hash, depth, depth_max,score):
    BASE_URL = 'https://blockstream.info/api/'
    url = BASE_URL + 'tx/' + str(tx_hash) 
    
    response = requests.get(url)
    if response.status_code == 200:
        tx = response.json()
    
    inputs = tx["vin"]
    print(f"depth {depth}, {len(inputs)} inputs")
    if len(inputs) <= 40:
        for j in range (len(inputs)) : 
            addr =  inputs[j]['prevout']["scriptpubkey_address"]
            if addr in data['address'].values :
                print(f"illegal adress found upstream {addr} at depth {depth}")
                for j in range (len(tx['vout'])):
                    if tx['vin'][j]['prevout']['scriptpubkey_address'] == addr:
                        satoshi_in = tx['vin'][j]['prevout']['value']
                score_addr = satoshi_in/depth
                print(score_addr)
                depth = depth_max
            else :
                score_addr = 0
            score = max(score,score_addr)

            #print(f"{tx['txid']} --> {tx_i['txid']}")
            if  depth < depth_max:
                #recursive_search(tx_i['txid'],depth, depth_max, score)
                score = max(score,recursive_upstream_search(inputs[j]['txid'], depth+1, depth_max, score))

            #print(f"{addr} has not reused {tx['txid']}")
    return score

In [236]:
score = recursive_upstream_search('3f4b0969254776fcd93a28bea2bc33982db6e03ba4c6f7a9a453bf631d5cf240', 1, 4, 0)
score

depth 1, 1 inputs
depth 2, 37 inputs
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 2
8701663.5
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal

8701663.5

In [238]:
score = recursive_upstream_search('c277a6c3d2c32b2c75ceebaca140b2e9ca9a31b32f464b64a4b11ab858b81708', 1, 4, 0)
score

depth 1, 1 inputs
depth 2, 25 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 3 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 9 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 18 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 1 inputs
depth 3, 1 inputs
depth 4, 37 inputs
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4vNYEAgA at depth 4
4350831.75
illegal adress found upstream 1Ee9ZiZkmygAXUiyeYKRSA3tLe4v

4350831.75

In [31]:
list(map((lambda x:x["txid"]),get_address_txs("18yXaf6HErMp26BEZwe8rCRJJMH5XMcvve")))

['10e538420b5d572436df0895db9c4168481b7d313c71cb9351ba30fc4f87b233',
 '7660c3c260148a231455c5978762a935b048860bf740da469bf1f247b9bd5518',
 '512949fa7b4b3b25038a2121eab66505b61547e67040f6420833d90b3648fa23',
 'a45aabb7470c61c05d195758b8746d7bd1445a756deac713264a0e32936c664b']

In [32]:
list(map((lambda x:x["txid"]),get_address_txs("1JwWLvfCvkFGepfsyYLLBES8LRVB3N4Dey")))

['7660c3c260148a231455c5978762a935b048860bf740da469bf1f247b9bd5518',
 'a45aabb7470c61c05d195758b8746d7bd1445a756deac713264a0e32936c664b',
 'b6b62b8b61b3838da15198541dff98fa40af38234760595acbf6212b27da9072',
 '03588a105bf61f628502ab1b69b16ba585bb964ec1c2d857f5784de16fd24381']

In [13]:
score = recursive_upstream_search('e377ac9333e4527fa86d4e096525f7ca81e9fd6212237d8601a0ccae916de23c', 1, 3, 0)
score

depth 1, 4 inputs
depth 2, 2 inputs
depth 3, 1 inputs
depth 3, 26 inputs
depth 2, 4 inputs
depth 3, 4 inputs
depth 3, 1 inputs
depth 3, 15 inputs
depth 3, 2 inputs
depth 2, 1 inputs
depth 3, 1 inputs
depth 2, 9 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs
depth 3, 1 inputs


0